In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
#import sys, time, datetime, random, math, pickle
from sklearn.linear_model import LogisticRegression
# auc and roc
from sklearn.metrics import roc_auc_score, roc_curve
# Balanced accuracy
from sklearn.metrics import balanced_accuracy_score
# Confusion matrix
from sklearn.metrics import confusion_matrix
# Classification report
from sklearn.metrics import recall_score, balanced_accuracy_score, f1_score, roc_auc_score
# SVG
#from IPython.display import SVG, display
# Save sklearn model
import joblib
#import warnings
import xgboost as xgb
#import gc
import torchviz


import lookup_dicts

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Machine Learning based Automatic Quality Control of Climatological Greenlantic Sensor data

## 1. Handling Data

In [2]:
# Load dictionaries
ELEM_DICT = lookup_dicts.ELEM_DICT
INDEX_TO_FEATURE = lookup_dicts.INDEX_TO_FEATURE
PATH_DATA = lookup_dicts.PATH_DATA

In [3]:
# Load training data
X_train = torch.load(PATH_DATA['train']['X']).float()
y_train = torch.load(PATH_DATA['train']['y']).float().squeeze()
# Load validation data
X_val = torch.load(PATH_DATA['val']['X']).float()
y_val = torch.load(PATH_DATA['val']['y']).float().squeeze()
# Load test data
X_test = torch.load(PATH_DATA['test']['X']).float()
y_test = torch.load(PATH_DATA['test']['y']).float().squeeze()


In [4]:
# Assert that X_train[0,3] <= X_train[-1,3] <= X_val[0,3] <= X_val[-1,3] <= X_test[0,3] <= X_test[-1,3]
num_features = X_train.shape[1]
num_classes = y_train.shape[1] if len(y_train.shape) > 1 else 1
num_workers = os.cpu_count()
output_size = num_classes

assert (
    X_train[0, 3] <= X_train[-1, 3]
    <=  X_val[0, 3] <= X_val[ -1,3]
    <= X_test[0, 3] <= X_test[-1,3]), "Data is not sorted by time."
print(f'X_train from {str(int(X_train[0,3]))} to {str(int(X_train[-1,3]))}, and X_val from {str(int(X_val[0,3]))} to {str(int(X_val[-1,3]))}, and X_test from {str(int(X_test[0,3]))} to {str(int(X_test[-1,3]))}.')
print(f'X_train.shape = {X_train.shape}.')
print(f'num_features = {num_features}.')
print(f'num_classes = {num_classes}.')
print(f'output_size := {output_size}.')
print(f'num_workers := {num_workers}.')
print(f'DEVICE := {DEVICE}.')


X_train from 1958 to 2011, and X_val from 2011 to 2017, and X_test from 2017 to 2022.
X_train.shape = torch.Size([41176872, 7]).
num_features = 7.
num_classes = 1.
output_size := 1.
num_workers := 12.
DEVICE := cuda.


In [5]:
def balanced_accuracy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (tp/(tp+fn) + tn/(tn+fp))/2.0


## 3 models: 1. Logistic Regression, 2. XGBoost, 3. Multi-Layer Perceptron

### 1. Logistic Regression

In [6]:
# Make class that inherits from sklearn's LogisticRegression and overrides the fit method
class LogisticRegression(LogisticRegression):
    def __init__(self):
        super().__init__()
        self.class_weight = 'balanced'
        self.max_iter = 1000
        self.n_jobs = -1
        self.verbose = 1
    def fit(self, X, y, sample_weight=None, **kwargs):
        # Save new model
        print('Saving new model...')
        self.X = X.to('cpu').numpy()
        self.y = y.to('cpu').numpy().squeeze()
        self.sample_weight = sample_weight.to('cpu').numpy().squeeze() if sample_weight is not None else None
        logreg = super().fit(self.X, self.y, sample_weight=self.sample_weight)
        PATH_TO_SAVE = 'models/checkpoint/logreg.pkl'
        if not os.path.exists(os.path.dirname(PATH_TO_SAVE)):
            os.makedirs(os.path.dirname(PATH_TO_SAVE))
        # Use joblib to save the model
        joblib_file = PATH_TO_SAVE
        joblib.dump(logreg, joblib_file)
        return logreg
    def predict(self, X):
        return super().predict(X)
    def predict_proba(self, X):
        return super().predict_proba(X)
    def save_predictions(self, X, y, data_set='test'):
        if type(X) == torch.Tensor:
            X = X.to('cpu').numpy()
        if type(y) == torch.Tensor:
            y = y.to('cpu').numpy().squeeze()
        y_pred = self.predict(X)
        y_pred_proba = self.predict_proba(X)
        y_pred_proba = y_pred_proba[:, 1]
        PATH_TO_SAVE = f'data/{data_set}/preds/logreg/'
        if not os.path.exists(os.path.dirname(PATH_TO_SAVE)):
            os.makedirs(os.path.dirname(PATH_TO_SAVE))
        np.save(PATH_TO_SAVE + 'y_pred.npy', y_pred)
        np.save(PATH_TO_SAVE + 'y_pred_proba.npy', y_pred_proba)
        np.save(PATH_TO_SAVE + 'y_true.npy', y)
# Initialize the model
logreg = LogisticRegression()

### 2. XGBoost


In [7]:
# X_train, y_train, X_val, y_val, X_test, y_test in numpy format
X_train_np, y_train_np, X_val_np, y_val_np, X_test_np, y_test_np = X_train.to('cpu').numpy(), y_train.to(
    'cpu').numpy(), X_val.to('cpu').numpy(), y_val.to('cpu').numpy(), X_test.to('cpu').numpy(), y_test.to('cpu').numpy()


def make_dmatrices_weighted(X_train, y_train, X_val, y_val, X_test, y_test, use_same_dmatrices=True):
    # Check if dtrain, dval, dtest already exist
    if (use_same_dmatrices and (os.path.exists('data/dmatrices/dtrain.buffer') and os.path.exists('data/dmatrices/dval.buffer') and os.path.exists('data/dmatrices/dtest.buffer'))):
        print('Loading dmatrices...')
        dtrain = xgb.DMatrix('data/dmatrices/dtrain.buffer', nthread=-1)
        dval = xgb.DMatrix('data/dmatrices/dval.buffer', nthread=-1)
        dtest = xgb.DMatrix('data/dmatrices/dtest.buffer', nthread=-1)
        weights = dtrain.get_weight().squeeze()
        # Get unique weights
        weights = np.unique(weights)
        print(f'Using weights: w_0_train = {weights[0]}, w_1_train = {weights[1]}')
        return dtrain, dval, dtest
    print('Creating dmatrices...')
    N_0 = np.sum(y_train == 0)
    N_1 = np.sum(y_train == 1)
    w_0_train = (2*N_1) / N_0
    w_1_train = N_0 / (N_1*2)
    print(f'Using weights: w_0_train = {w_0_train}, w_1_train = {w_1_train}')
    weights_train = np.zeros(len(y_train))
    weights_train[y_train == 0] = w_0_train
    weights_train[y_train == 1] = w_1_train
    weights_val = np.zeros(len(y_val))
    weights_val[y_val == 0] = w_0_train
    weights_val[y_val == 1] = w_1_train
    weights_test = np.zeros(len(y_test))
    weights_test[y_test == 0] = w_0_train
    weights_test[y_test == 1] = w_1_train
    dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights_train)
    dval = xgb.DMatrix(X_val, label=y_val, weight=weights_val)
    dtest = xgb.DMatrix(X_test, label=y_test, weight=weights_test)
    # Check if folder exists for dmatrices
    if not os.path.exists('data/dmatrices/'):
        os.makedirs('data/dmatrices/')
    # Save dmatrices
    dtrain.save_binary('data/dmatrices/dtrain.buffer')
    dval.save_binary('data/dmatrices/dval.buffer')
    dtest.save_binary('data/dmatrices/dtest.buffer')
    return dtrain, dval, dtest

dtrain, dval, dtest = make_dmatrices_weighted(X_train_np, y_train_np, X_val_np, y_val_np, X_test_np, y_test_np, use_same_dmatrices=False)

positives = np.sum(dtrain.get_label())  # The number of positive samples
total = np.shape(dtrain.get_label())[0]  # The total number of samples
negatives = total - positives  # The number of negative samples
scale_pos_weight = negatives / (2*total)  # The ratio of negative to positive samples
print(f'Using scale_pos_weight = {scale_pos_weight}')

Creating dmatrices...
Using weights: w_0_train = 63.51287366892007, w_1_train = 0.01574483946692131
Using scale_pos_weight = 0.01526492833161295


In [8]:
params = {
    'objective': 'binary:logistic',
    'tree_method': 'gpu_hist',
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'scale_pos_weight': scale_pos_weight,
    'seed': 42,
    'verbosity': 1,
    'n_jobs': -1,
    'eval_metric': 'logloss',
    'predictor': 'gpu_predictor',
    'gpu_id': 0
}

In [9]:
# Load model
def xgb_model_loader(early_stopping_rounds=2000):
    if not os.path.exists('models/checkpoint/xgb_model.model'):
        print('Training model...')
        xgb_model = xgb.train(params, dtrain, num_boost_round=10000, evals=[(dval, 'val')], early_stopping_rounds=early_stopping_rounds, verbose_eval=100)
        xgb_model.save_model('models/checkpoint/xgb_model.model')
    else:
        print('Loading model...')
        xgb_model = xgb.Booster()
        xgb_model.load_model('models/checkpoint/xgb_model.model')
    assert xgb_model is not None, 'Model not loaded'
    return xgb_model

def xgb_model_predictions(data_set='test', new_preds=False):
    # Load predictions if possible
    if os.path.exists(f'data/{data_set}/preds/xgboost/y_pred.npy') and not new_preds:
        print('Loading predictions...')
        y_pred = np.load(f'data/{data_set}/preds/xgboost/y_pred.npy')
        y_pred_proba = np.load(f'data/{data_set}/preds/xgboost/y_pred_proba.npy')
        y_true = np.load(f'data/{data_set}/preds/xgboost/y_true.npy')
        return y_pred, y_pred_proba, y_true
    # Load model
    xgb_model = xgb_model_loader()
    # Make predictions in the data set
    if data_set == 'test':
        d = dtest
    elif data_set == 'val':
        d = dval
    elif data_set == 'train':
        d = dtrain
    else:
        raise ValueError('data_set must be one of: train, val, test')
    y_pred_proba = xgb_model.predict(d)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    y_true = d.get_label()
    # Save predictions
    PATH_TO_SAVE = f'data/{data_set}/preds/xgboost/'
    if not os.path.exists(os.path.dirname(PATH_TO_SAVE)):
        os.makedirs(os.path.dirname(PATH_TO_SAVE))
    np.save(PATH_TO_SAVE + 'y_pred.npy', y_pred)
    np.save(PATH_TO_SAVE + 'y_pred_proba.npy', y_pred_proba)
    np.save(PATH_TO_SAVE + 'y_true.npy', y_true)
    return y_pred, y_pred_proba, y_true

### 3. Multi-Layer Perceptron

In [10]:
# Create the datasets
class MLPDataset(Dataset):
    def __init__(self, X, y, device=DEVICE):
        self.X = X.to(device)
        self.y = y.squeeze().to(device)
        self.n = X.shape[0]
        # Calculate weights used for BCEWithLogitsLoss
        self.zeroes = torch.sum(self.y == 0)
        self.ones = torch.sum(self.y == 1)
        self.weights = torch.zeros(self.n)
        self.weights[self.y == 0] = (self.ones) / (self.n)
        self.weights[self.y == 1] = (self.zeroes) / (self.n)
        # Normalize weights
        self.weights = self.weights / torch.sum(self.weights)
        # Move to device
        self.weights = self.weights.to(device)

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.weights[idx]

    def get_X(self):
        return self.X

    def get_y(self):
        return self.y

    def get_weights(self):
        return self.weights

    def get_X_y_idx(self, idx):
        return self.X[idx], self.y[idx]

    def get_X_y_weights_idx(self, idx):
        return self.X[idx], self.y[idx], self.weights[idx]

# Create the dataloaders
class MLPDataLoader():
    def __init__(self, dataset, batch_size, device=DEVICE, num_workers=num_workers, pin_memory=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.n = len(dataset)
        self.m_batches = self.n // self.batch_size
        self.indices = np.arange(self.n)
        self.i = 0
        self.M = self.m_batches * self.batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            self.i = 0
            raise StopIteration
        else:
            idx = self.indices[self.i:self.i+self.batch_size]
            self.i += self.batch_size
            return self.dataset.get_X_y_weights_idx(idx)

    def __len__(self):
        return self.m_batches

    def __getitem__(self, idx):
        return self.dataset.get_X_y_weights_idx(self.indices[idx*self.batch_size:(idx+1)*self.batch_size])


In [11]:
PATH_TO_MLP = 'models/checkpoint/multilayerPerceptron.ckpt'

In [12]:
# Loss function that is weighted by the class weights
def weighted_bce_loss(y_pred, y_true, weights):
    return F.binary_cross_entropy_with_logits(y_pred, y_true, weights, reduction='sum')
loss_fn = weighted_bce_loss

In [13]:
# The balanced accuracy metric
def balanced_accuracy(y_pred, y_true):
    '''Calculates the balanced accuracy'''
    y_pred = torch.round(torch.sigmoid(y_pred))
    tp = torch.sum(y_pred * y_true)
    tn = torch.sum((1 - y_pred) * (1 - y_true))
    fp = torch.sum(y_pred * (1 - y_true))
    fn = torch.sum((1 - y_pred) * y_true)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    return (tpr + tnr) / 2

acc = balanced_accuracy

In [14]:
hidden_size = 2**(num_features)
num_epochs = 2**(num_features+1)
learning_rate = 10**-5
batch_size = 2**20
weight_decay = learning_rate * 0.1
momentum = 0.9
factor = 0.1

print(f'hidden_size := {hidden_size}.')
print(f'num_epochs := {num_epochs}.')
print(f'learning_rate := {learning_rate}.')
print(f'batch_size := {batch_size}.')
print(f'weight_decay := {weight_decay}.')
print(f'momentum := {momentum}.')
print(f'factor := {factor}.')

hidden_size := 128.
num_epochs := 256.
learning_rate := 1e-05.
batch_size := 1048576.
weight_decay := 1.0000000000000002e-06.
momentum := 0.9.
factor := 0.1.


In [15]:
# The multi-layer perceptron
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device):
        super(MLP, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size, device=device)
        self.fc2 = nn.Linear(hidden_size, hidden_size//2, device=device)
        self.fc3 = nn.Linear(hidden_size//2, hidden_size//4, device=device)
        self.fc4 = nn.Linear(hidden_size//4, output_size, device=device)

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out.squeeze()

    def predict(self, x):
        return torch.sigmoid(self.forward(x))

    def predict_class(self, x):
        return torch.round(self.predict(x)).squeeze()


# The model: multilayerPerceptron
multilayerPerceptron = MLP(input_size=num_features, hidden_size=hidden_size, output_size=num_classes, device=DEVICE)

# The optimizer
optimizer = torch.optim.SGD(multilayerPerceptron.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

# The scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=factor, patience=50, verbose=True)

# Define the dataloaders
train_dataloader = MLPDataLoader(MLPDataset(X_train, y_train), batch_size, device=DEVICE)
val_dataloader = MLPDataLoader(MLPDataset(X_val, y_val), batch_size, device=DEVICE)
test_dataloader = MLPDataLoader(MLPDataset(X_test, y_test), batch_size, device=DEVICE)

# Train the model
def train(model, optimizer, loss_fn, scheduler, train_dataloader, val_dataloader, num_epochs, device):
    train_losses = []
    val_losses = []
    train_accs = []
    best_train_acc = 0
    val_accs = []
    best_val_acc = 0
    best_acc_epoch = 0
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        train_acc = 0
        for X, y, weights in train_dataloader:
            X = X.to(device)
            y = y.to(device)
            weights = weights.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y, weights)
            train_loss += loss.item()
            train_acc += acc(y_pred, y).item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = train_loss / len(train_dataloader)
        train_acc = train_acc / len(train_dataloader)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        # Evaluate
        model.eval()
        val_loss = 0
        val_acc = 0
        for X, y, weights in val_dataloader:
            X = X.to(device)
            y = y.to(device)
            weights = weights.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y, weights)
            val_loss += loss.item()
            val_acc += acc(y_pred, y).item()
        val_loss = val_loss / len(val_dataloader)
        val_acc = val_acc / len(val_dataloader)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        # Print
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        # Update the scheduler
        scheduler.step(val_acc)
        # Save the model if it is the best
        if val_acc > best_val_acc and train_acc > 0.5:
            best_val_acc = val_acc
            best_train_acc = train_acc
            best_acc_epoch = epoch
            torch.save(model.state_dict(), PATH_TO_MLP)
            print(f'Best model saved at epoch {epoch+1}.')
    print(f'Best val acc: {best_val_acc:.4f} at epoch {best_acc_epoch+1}.')
    return train_losses, val_losses, train_accs, val_accs

#train_losses, val_losses, train_accs, val_accs = train(multilayerPerceptron, optimizer, loss_fn, scheduler, train_dataloader, val_dataloader, num_epochs, DEVICE)

In [16]:
def loadMLP(model=multilayerPerceptron, path=PATH_TO_MLP):
    if os.path.isfile(path):
        model.load_state_dict(torch.load(path))
        print(f'Model loaded from {path}.')
    else:
        print(f'No model found at {path}.')
        train_losses, val_losses, train_accs, val_accs = train(model, optimizer, loss_fn, scheduler, train_dataloader, val_dataloader, num_epochs, DEVICE)
    model.eval()
    return model

# Load the model
multilayerPerceptron = loadMLP(multilayerPerceptron, path='models/checkpoint/model.ckpt')

# Function to get (logits, preds, true_labels) pairs
def get_preds(model=multilayerPerceptron, dataloader=MLPDataLoader(MLPDataset(X_test, y_test), batch_size, device=DEVICE), data_set='test'):
    model.eval()
    logits = []
    preds = []
    true_labels = []
    with torch.no_grad():
        for X, y, weights in dataloader:
            X = X.to(DEVICE)
            y = y.to(DEVICE)
            weights = weights.to(DEVICE)
            y_pred = model(X)
            logits.append(y_pred)
            preds.append(torch.round(y_pred))
            true_labels.append(y)
    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    true_labels = torch.cat(true_labels, dim=0)
    # Save the predictions to files
    PATH_STORE_MLP_LOGITS = 'data/{data_set}/preds/mlp/mlp_preds_proba.pt'.format(data_set=dataloader.dataset.data_set)
    PATH_STORE_MLP_PREDS = 'data/{data_set}/preds/mlp/mlp_preds.pt'.format(data_set=dataloader.dataset.data_set)
    PATH_STORE_MLP_TRUE_LABELS = 'data/{data_set}/preds/mlp/mlp_true.pt'.format(data_set=dataloader.dataset.data_set)
    torch.save(logits, PATH_STORE_MLP_LOGITS)
    torch.save(preds, PATH_STORE_MLP_PREDS)
    torch.save(true_labels, PATH_STORE_MLP_TRUE_LABELS)
    return logits, preds, true_labels

Model loaded from models/checkpoint/model.ckpt.


In [17]:
# Print the model's architecture
print(multilayerPerceptron)
# Plot the model's architecture
PATH_TO_MLP_ARCHITECTURE = 'figs/mlp_architecture'
torchviz.make_dot(multilayerPerceptron(X_train[0:1].to(DEVICE)), params=dict(
    multilayerPerceptron.named_parameters()), show_attrs=True, show_saved=True).render(PATH_TO_MLP_ARCHITECTURE, format="png", cleanup=True, quiet=False)

MLP(
  (relu): ReLU()
  (fc1): Linear(in_features=7, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


'figs/mlp_architecture.png'

In [18]:
# Function to load predictions if new_preds is False
def load_mlp_preds(data_set='test', new_preds=False):
    if new_preds:
        logits, preds, true_labels = get_preds(multilayerPerceptron, MLPDataLoader(MLPDataset(X_test, y_test), batch_size, device=DEVICE, data_set='test'))
    else:
        PATH_STORE_MLP_LOGITS = 'data/{data_set}/preds/mlp/mlp_preds_proba.pt'.format(data_set=data_set)
        PATH_STORE_MLP_PREDS = 'data/{data_set}/preds/mlp/mlp_preds.pt'.format(data_set=data_set)
        PATH_STORE_MLP_TRUE_LABELS = 'data/{data_set}/preds/mlp/mlp_true.pt'.format(data_set=data_set)
        logits = torch.load(PATH_STORE_MLP_LOGITS)
        preds = torch.load(PATH_STORE_MLP_PREDS)
        true_labels = torch.load(PATH_STORE_MLP_TRUE_LABELS)
    return logits, preds, true_labels

In [19]:
def get_predictions(model='logreg', new_preds=False):
    if model == 'logreg':
        # Get the predictions if they already exist
        if os.path.exists('data/test/preds/logreg/y_pred.npy') and os.path.exists('data/val/preds/logreg/y_pred.npy') and os.path.exists('data/train/preds/logreg/y_pred.npy') and not new_preds:
            print('Loading predictions...')
            y_pred_test = np.load('data/test/preds/logreg/y_pred.npy')
            y_pred_test_proba = np.load('data/test/preds/logreg/y_pred_proba.npy')
            y_test_true = np.load('data/test/preds/logreg/y_true.npy')
            y_pred_val = np.load('data/val/preds/logreg/y_pred.npy')
            y_pred_val_proba = np.load('data/val/preds/logreg/y_pred_proba.npy')
            y_val_true = np.load('data/val/preds/logreg/y_true.npy')
            y_pred_train = np.load('data/train/preds/logreg/y_pred.npy')
            y_pred_train_proba = np.load('data/train/preds/logreg/y_pred_proba.npy')
            y_train_true = np.load('data/train/preds/logreg/y_true.npy')

        # Otherwise, fit the model and save the predictions
        else:
            print('Fitting model...')
            logreg = LogisticRegression()
            logreg.fit(X_train, y_train)
            logreg.save_predictions(X_test, y_test, data_set='test')
            logreg.save_predictions(X_val, y_val, data_set='val')
            logreg.save_predictions(X_train, y_train, data_set='train')
            y_pred_test = logreg.predict(X_test)
            y_pred_test_proba = logreg.predict_proba(X_test)
            y_pred_test_proba = y_pred_test_proba[:, 1]
            y_test_true = y_test
            y_pred_val = logreg.predict(X_val)
            y_pred_val_proba = logreg.predict_proba(X_val)
            y_pred_val_proba = y_pred_val_proba[:, 1]
            y_val_true = y_val
            y_pred_train = logreg.predict(X_train)
            y_pred_train_proba = logreg.predict_proba(X_train)
            y_pred_train_proba = y_pred_train_proba[:, 1]
            y_train_true = y_train
    if model == 'xgboost':
        y_pred_test, y_pred_test_proba, y_test_true = xgb_model_predictions(data_set='test', new_preds=new_preds)
        y_pred_val, y_pred_val_proba, y_val_true = xgb_model_predictions(data_set='val', new_preds=new_preds)
        y_pred_train, y_pred_train_proba, y_train_true = xgb_model_predictions(data_set='train', new_preds=new_preds)
    if model == 'mlp':
        y_pred_test_proba, y_pred_test, y_test_true = load_mlp_preds(data_set='test', new_preds=new_preds)
        y_pred_val_proba, y_pred_val, y_val_true = load_mlp_preds(data_set='val', new_preds=new_preds)
        y_pred_train_proba, y_pred_train, y_train_true = load_mlp_preds(data_set='train', new_preds=new_preds)
    train_pred, val_pred, test_pred = (y_pred_train, y_pred_train_proba, y_train_true), (y_pred_val, y_pred_val_proba, y_val_true), (y_pred_test, y_pred_test_proba, y_test_true)
    return train_pred, val_pred, test_pred

In [20]:
def cmatrix_plot(model_name='logreg', data_set='test',reload_preds=False):
    assert data_set in ['train', 'val', 'test'], 'data_set must be one of "train", "val", or "test"'
    index_of_interest = 0 if data_set == 'train' else 1 if data_set == 'val' else 2 if data_set == 'test' else None
    y_pred, _, y_true = get_predictions(
        model=model_name, new_preds=reload_preds)[index_of_interest]
    if torch.is_tensor(y_pred):
        y_pred = y_pred.detach().cpu().numpy()
    if torch.is_tensor(y_true):
        y_true = y_true.detach().cpu().numpy()
    cm1 = confusion_matrix(y_true, y_pred, normalize=None, labels=[0, 1])
    cm2 = confusion_matrix(y_true, y_pred, normalize='true', labels=[0, 1])
    fig, ax = plt.subplots(1, 2, figsize=(10, 4), dpi=300)
    sns.heatmap(cm1, annot=True, fmt='d', ax=ax[0], cmap='Blues')
    sns.heatmap(cm2, annot=True, fmt='.2f', ax=ax[1], cmap='Blues')
    # Model name
    if model_name == 'logreg':
        model_name = 'Logistic Regression'
    elif model_name == 'xgboost':
        model_name = 'XGBoost'
    elif model_name == 'mlp':
        model_name = 'Multilayer Perceptron'
    fig.suptitle(model_name, fontsize=16)
    # Set titles and labels
    ax[0].set_title('Confusion Matrix')
    ax[1].set_title('Normalized Confusion Matrix')
    ax[0].set_xlabel('Predicted')
    ax[0].set_ylabel('True')
    ax[1].set_xlabel('Predicted')
    ax[1].set_ylabel('True')
    # Set ticks
    ax[0].set_xticklabels(['Exclude', 'Include'])
    ax[0].set_yticklabels(['Exclude', 'Include'])
    ax[1].set_xticklabels(['Exclude', 'Include'])
    ax[1].set_yticklabels(['Exclude', 'Include'])
    plot_name = '{}_{}_cmatrix.png'.format(model_name, data_set)
    plt.savefig('figs/{}'.format(plot_name))
    plt.show()

# Roc curve
def roc_plot(model_name='logreg', data_set='test'):
    assert data_set in [
        'train', 'val', 'test'], 'data_set must be one of "train", "val", or "test"'
    if data_set == 'train':
        _, y_pred_proba, y_true = get_predictions(model=model_name)[0]
    elif data_set == 'val':
        _, y_pred_proba, y_true = get_predictions(model=model_name)[1]
    elif data_set == 'test':
        _, y_pred_proba, y_true = get_predictions(model=model_name)[2]
    # Compute the roc curve
    if torch.is_tensor(y_pred_proba):
        y_pred_proba = y_pred_proba.cpu().detach().numpy()
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().detach().numpy()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    # Compute the auc
    auc = roc_auc_score(y_true, y_pred_proba)
    # Plot the roc curve
    if model_name == 'logreg':
        model_name = 'Logistic Regression'
    elif model_name == 'xgboost':
        model_name = 'XGBoost'
    elif model_name == 'mlp':
        model_name = 'Multilayer Perceptron'
    plt.figure(figsize=(6,3), dpi=300)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.4f})')
    plt.plot([np.min(fpr), np.max(fpr)], [np.min(fpr), np.max(fpr)], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plot_name = '{}_{}_roc.png'.format(model_name, data_set)
    plt.savefig('figs/{}'.format(plot_name))
    plt.show()


# Plot the confusion matrix for the test set for the logistic regression model
#cmatrix_plot(model_name='mlp', data_set='test', reload_preds=False)

# Plot the roc curve for the test set for the logistic regression model
#roc_plot(model_name='mlp', data_set='test')


In [21]:
# plot roc curve for all models on test set in one plot and save to file
def roc_plot_all_models():
    plt.figure(figsize=(6,3), dpi=300)
    for model in ['logreg', 'xgboost', 'mlp']:
        _, y_pred_proba, y_true = get_predictions(model=model)[2]
        # Check if y_pred_proba or y_true is on GPU
        try:
            if y_pred_proba.is_cuda:
                y_pred_proba = y_pred_proba.cpu().detach().numpy()
                y_true = y_true.cpu().detach().numpy()
        except:
            pass
        # Compute the roc curve
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
        # Compute the auc
        auc = roc_auc_score(y_true, y_pred_proba)
        # Plot the roc curve
        if model == 'logreg':
            model_name = 'Logistic Regression'
        elif model == 'xgboost':
            model_name = 'XGBoost'
        elif model == 'mlp':
            model_name = 'Multilayer Perceptron'
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.savefig('figs/all_models_test_roc.png')
    plt.show()

#roc_plot_all_models()

In [22]:
# Cmatrix for all models on test set in one plot and save to file
def cmatrix_plot_all_models():
    fig, ax = plt.subplots(2, 3, figsize=(10, 7), dpi=300)
    for i, model in enumerate(['logreg', 'xgboost', 'mlp']):
        y_pred, _, y_true = get_predictions(model=model)[2]
        # Check if y_pred_proba or y_true is on GPU
        try:
            if y_pred.is_cuda:
                y_pred = y_pred.cpu().detach().numpy()
                y_true = y_true.cpu().detach().numpy()
        except:
            pass
        cm1 = confusion_matrix(y_true, y_pred, normalize=None, labels=[0, 1])
        cm2 = confusion_matrix(y_true, y_pred, normalize='true', labels=[0, 1])
        sns.heatmap(cm1, annot=True, fmt='d', ax=ax[0, i], cmap='Blues')
        sns.heatmap(cm2, annot=True, fmt='.2f', ax=ax[1, i], cmap='Blues')
        # Model name
        if model == 'logreg':
            model_name = 'Logistic Regression'
        elif model == 'xgboost':
            model_name = 'XGBoost'
        elif model == 'mlp':
            model_name = 'Multilayer Perceptron'
        # Set titles and labels
        # Set the overall title
        fig.suptitle('Confusion Matrix', fontsize=16)
        ax[0, i].set_title(f'{model_name}')
        ax[1, i].set_title(f'{model_name}, Normalized')
        ax[0, i].set_xlabel('Predicted')
        ax[0, i].set_ylabel('True')
        ax[1, i].set_xlabel('Predicted')
        ax[1, i].set_ylabel('True')
        # Set ticks
        ax[0, i].set_xticklabels(['Exclude', 'Include'])
        ax[0, i].set_yticklabels(['Exclude', 'Include'])
        ax[1, i].set_xticklabels(['Exclude', 'Include'])
        ax[1, i].set_yticklabels(['Exclude', 'Include'])
    plt.tight_layout()
    plt.savefig('figs/all_models_test_cmatrix.png')
    plt.show()

#cmatrix_plot_all_models()

In [23]:
# Plot the feature importance for the xgboost model
def xgboost_feature_importance():
    # Load the model
    model = xgb.XGBClassifier()
    model.load_model('models/checkpoint/xgb_model.model')

    # Load labels from lookup_dict.py
    labels = lookup_dicts.INDEX_TO_FEATURE
    labels = [labels[i] for i in range(len(labels))]
    # Get the feature importance
    importance = model.get_booster().get_score(importance_type='gain')

    # Get the feature importance
    importance = model.get_booster().get_score(importance_type='gain')
    # Sort the feature importance
    importance = {labels[int(k[1:])]: v for k, v in importance.items()}
    # Plot the feature importance
    plt.figure(figsize=(6,3), dpi=300)
    plt.bar(importance.keys(), importance.values())
    plt.xticks(rotation=45)
    plt.ylabel('Feature Importance')
    plt.title('Feature Importance for XGBoost Model')
    plt.savefig('figs/xgboost_feature_importance.png')
    plt.show()

#xgboost_feature_importance()

In [24]:
# Plot the feature importance for the logistic regression model
def logreg_feature_importance():
    # Load the model using joblib
    model = joblib.load('models/checkpoint/logreg.pkl')
    # Load labels from lookup_dict.py
    labels = lookup_dicts.INDEX_TO_FEATURE
    labels = [labels[i] for i in range(len(labels))]
    # Get the feature importance
    importance = model.coef_[0]
    # Sort the feature importance
    importance = {labels[i]: importance[i] for i in range(len(importance))}
    importance = {k: v for k, v in sorted(importance.items(), key=lambda item: item[1])}
    # Plot the feature importance
    plt.figure(figsize=(6,3), dpi=300)
    plt.bar(importance.keys(), importance.values())
    plt.xticks(rotation=45)
    plt.ylabel('Feature Importance')
    plt.title('Feature Importance for Logistic Regression Model')
    plt.savefig('figs/logreg_feature_importance.png')
    plt.show()

#logreg_feature_importance()

In [25]:
# for all models print the TPR & TNR & Balanced Accuracy  & F1 & AUC in a pandas dataframe and print it

def print_metrics():
    # Create a dataframe to store the metrics
    metrics = pd.DataFrame(columns=['Model', 'TPR', 'TNR', 'Balanced Accuracy', 'F1', 'AUC'])
    for i, model in enumerate(['logreg', 'xgboost', 'mlp']):
        y_pred, y_pred_proba, y_true = get_predictions(model=model)[2]
        # Check if y_pred_proba or y_true is on GPU
        try:
            if y_pred_proba.is_cuda or y_true.is_cuda or y_pred.is_cuda:
                y_pred_proba = y_pred_proba.cpu().detach().numpy()
                y_pred = y_pred.cpu().detach().numpy()
                y_true = y_true.cpu().detach().numpy()
        except:
            pass
        # Compute the metrics
        tpr = recall_score(y_true, y_pred)
        tnr = recall_score(y_true, y_pred, pos_label=0)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_proba)
        # Round the metrics to 3 decimal places
        tpr = round(tpr, 3)
        tnr = round(tnr, 3)
        balanced_accuracy = round(balanced_accuracy, 3)
        f1 = round(f1, 3)
        auc = round(auc, 3)
        # Add the metrics to the dataframe
        if model == 'logreg':
            model_name = 'Logistic Regression'
        elif model == 'xgboost':
            model_name = 'XGBoost'
        elif model == 'mlp':
            model_name = 'Multilayer Perceptron'
        metrics.loc[i] = [model_name, tpr, tnr, balanced_accuracy, f1, auc]
    # Print the metrics
    print(metrics)
    return metrics

metrics = print_metrics()

Loading predictions...
Loading predictions...
Loading predictions...
Loading predictions...
                   Model    TPR    TNR  Balanced Accuracy     F1    AUC
0    Logistic Regression  0.626  0.673              0.650  0.757  0.718
1                XGBoost  0.864  0.605              0.735  0.910  0.790
2  Multilayer Perceptron  0.996  0.699              0.848  0.985  0.822
